# Wandb Integration

In [ ]:
pip install --upgrade wandb

In [ ]:
!wandb login dd888f73500a67fc53f9191092b22f3946ac0e02

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Init wandb
import wandb
from wandb.keras import WandbCallback

wandb.init(project="assignment-2-Part-A", entity="ravi-kumar")

# Declare Hyperparameters

# Imports

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.preprocessing import image_dataset_from_directory

# Loading Data

In [ ]:
def load_data():
  with tf.device('/device:GPU:0'):
    train_dir='/content/drive/MyDrive/Deep Learning/inaturalist_12K/train'
    test_dir='/content/drive/MyDrive/Deep Learning/inaturalist_12K/val'
    train_data = image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="categorical",
        class_names=None,
        color_mode="rgb",
        batch_size=32,
        image_size=(256, 256),
        shuffle=True,
        seed=0,
        validation_split=0.1,
        subset="training",
        interpolation="bilinear",
        follow_links=False,
    )
    val_data = image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="categorical",
        class_names=None,
        color_mode="rgb",
        batch_size=32,
        image_size=(256, 256),
        shuffle=True,
        seed=0,
        validation_split=0.1,
        subset="validation",
        interpolation="bilinear",
        follow_links=False,
    )
    test_data = image_dataset_from_directory(
        test_dir,
        labels="inferred",
        label_mode="categorical",
        class_names=None,
        color_mode="rgb",
        batch_size=32,
        image_size=(256, 256),
        shuffle=True,
        seed=0,
        interpolation="bilinear",
        follow_links=False,
    )
  
  return train_data, val_data, test_data


In [ ]:
with tf.device('/device:GPU:0'):
  train_data, val_data, test_data=load_data()
  class_names=train_data.class_names
  # train_data = train_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
  # val_data = val_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
  # test_data = test_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 10017 files belonging to 10 classes.
Using 9016 files for training.
Found 10017 files belonging to 10 classes.
Using 1001 files for validation.
Found 2001 files belonging to 10 classes.


# Data Augmentation

In [ ]:
with tf.device('/device:GPU:0'):
  DataAugmentation = Sequential(
    [tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(256, 256, 3)),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
      tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2)]
  )

# Transfer Learning [Q (1)]

In [ ]:
def get_model(model_name='InceptionV3', input_shape = (256, 256, 3)):
  with tf.device('/device:GPU:0'): 
    base_model = None

    if model_name == 'InceptionResNetV2':
      base_model = tf.keras.applications.InceptionResNetV2(input_shape=input_shape, include_top=False,  weights='imagenet')
    elif model_name == 'ResNet50':
      base_model = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False,  weights='imagenet')
    elif model_name == 'Xception':
      base_model = tf.keras.applications.Xception(input_shape=input_shape, include_top=False,  weights='imagenet')
    else:  
      base_model = tf.keras.applications.InceptionV3(input_shape=input_shape, include_top=False,  weights='imagenet')

    inputs = tf.keras.Input(shape=input_shape)
    x= DataAugmentation(inputs)
    x= tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=input_shape)(x)
    x= base_model(x, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x) 
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    return base_model, model

In [ ]:
# with tf.device('/device:GPU:0'):
#   input_shape = (256, 256, 3)
#   lr = 0.000005
#   epochs = 5
#   base_model, model = get_Model()

#   base_model.trainable = True

#   # model.summary()

#   model.compile(loss='categorical_crossentropy',
#                         optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
#                         metrics=['accuracy'])

#   model.fit(train_data, validation_data=val_data, epochs=epochs)

# Fine -Tuning [Q (2)]

In [ ]:
with tf.device('/device:GPU:0'):
  input_shape = (256, 256, 3)
  lr = 0.000005
  epochs = 5
  model_name = 'InceptionResNetV2'

  base_model, model = get_Model(model_name)

  base_model.trainable = False

  # model.summary()

  model.compile(loss='categorical_crossentropy',
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    metrics=['accuracy'])

  model.fit(train_data, validation_data=val_data, epochs=epochs)


# Comparing InceptionV3, InceptionResNetV2, ResNet50, Xception [Q (3)]

In [ ]:
def train():
  with tf.device('/device:GPU:0'):
    config_defaults = {
        'model_name' : 'InceptionV3',
        'learning_rate': 0.00001,
        'dropout' : 0.2,
        'epochs': 5
    }
    wandb.init(config=config_defaults)
    config= wandb.config

    input_shape = (256, 256, 3)
    lr = config.learning_rate
    epochs = config.epochs
    model_name = config.model_name

    base_model, model = get_model(model_name)

    base_model.trainable = True

    # model.summary()

    model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                      metrics=['accuracy'])

    model.fit(train_data, validation_data=val_data, epochs=epochs, callbacks=[WandbCallback()])


## Wandb Configurration

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model_name' : {
            'values': ['InceptionV3', 'InceptionResNetV2', 'ResNet50', 'Xception']
        },
        'learning_rate':{
            'values': [0.00001]
        },
        'dropout' :{
            'values': [0.2]
        },
        'epochs':{
            'values': [5]
        }

    }
}


In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ravi-kumar", project="assignment-2-Part-B")
wandb.agent(sweep_id, train)